## Partie 1 — Préparation du moteur de recherche et baseline de pertinence

In [1]:
from elasticsearch import Elasticsearch
from dotenv import load_dotenv
from openai import OpenAI
import pytrec_eval
import tiktoken
import mlflow
import json
import math
import os

### Data analysis

In [2]:
file_path = "data/wikipedia.json"

In [3]:
data = []

with open(file_path, "r", encoding="utf-8") as f :
    for line in f :
        line = line.strip()
        if line :  
            data.append(json.loads(line))

In [4]:
print(type(data))
print(len(data))

<class 'list'>
12020


In [23]:
all_keys = set()
for key in data :
    all_keys.update(key.keys())

In [24]:
all_keys

{'_id', 'content', 'games', 'skip', 'url'}

In [25]:
data[:2]

[{'_id': 'Pbq-UGAkFr',
  'url': 'https://en.wikipedia.org/wiki/Final_Fantasy_Adventure',
  'content': {'title': 'Final Fantasy Adventure',
   'summary': 'Final Fantasy Adventure, known in Japan as Seiken Densetsu: Final Fantasy Gaiden or simply Seiken Densetsu, and later released in Europe as Mystic Quest, is a 1991 action role-playing game developed and published by Square for the Game Boy. It is a spin-off of the Final Fantasy series and the first game in the Mana series. \nOriginally developed under the name Gemma Knights, it features gameplay roughly similar to that of the original The Legend of Zelda, but with the addition of role-playing statistical elements. A remake, Sword of Mana, was released for the Game Boy Advance in 2003, changing the plot and many gameplay aspects. A second remake was released on mobile phones in Japan which improved the graphics and music of the original version. A third remake, Adventures of Mana, was released for iOS, Android, and PlayStation Vita on 

In [26]:
def get_keys(obj, parent_key='') :
    keys = set()
    if isinstance(obj, dict):
        for k, v in obj.items() :
            full_key = f"{parent_key}.{k}" if parent_key else k
            keys.add(full_key)
            keys.update(get_keys(v, full_key))

    elif isinstance(obj, list) :
        for item in obj:
            keys.update(get_keys(item, parent_key))
    return keys

all_keys = get_keys(data)

In [27]:
for key in sorted(all_keys) :
    print(key)

_id
content
content.sections
content.sections.subsections
content.sections.subsections.subsections
content.sections.subsections.subsections.subsections
content.sections.subsections.subsections.subsections.subsections
content.sections.subsections.subsections.subsections.subsections.subsections
content.sections.subsections.subsections.subsections.subsections.text
content.sections.subsections.subsections.subsections.subsections.title
content.sections.subsections.subsections.subsections.text
content.sections.subsections.subsections.subsections.title
content.sections.subsections.subsections.text
content.sections.subsections.subsections.title
content.sections.subsections.text
content.sections.subsections.title
content.sections.text
content.sections.title
content.summary
content.title
games
skip
url


### Index es

In [28]:
es = Elasticsearch("http://localhost:9200")
index_name = "wikipedia"

In [29]:
print(es.info())

{'name': '921fa652c27e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'cInJFOj1Ttu98Lb6npU61g', 'version': {'number': '8.19.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '93788a8c2882eb5b606510680fac214cff1c7a22', 'build_date': '2025-07-23T22:10:18.138212839Z', 'build_snapshot': False, 'lucene_version': '9.12.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [30]:
if not es.indices.exists(index=index_name) :
    es.indices.create(index=index_name)

In [31]:
for doc in data:
    doc_id = doc.pop("_id", None)
    es.index(index="wikipedia", id=doc_id, document=doc)

print(f"Indexation finie : {len(data)} docs")

Indexation finie : 12020 docs


### Interrogation de la base (top 100)

In [ ]:
# Selection des champs pertinents
def get_text_fields(obj, parent_key=''):
    """
    Parcourt le dataset et retourne les key qui sont sous format str
    """
    text_fields = set()
    
    if isinstance(obj, dict) :
        for k, v in obj.items() :
            full_key = f"{parent_key}.{k}" if parent_key else k

            if isinstance(v, str):
                text_fields.add(full_key)

            elif isinstance(v, list) :
                if all(isinstance(i, str) for i in v) :
                    text_fields.add(full_key)
                else:
                    for item in v : 
                        text_fields.update(get_text_fields(item, full_key))

            elif isinstance(v, dict) :
                # print('test')
                text_fields.update(get_text_fields(v, full_key))
    
    return text_fields

In [33]:
text_fields = get_text_fields(data[0])
print("Champs contenant du texte :")
for f in sorted(text_fields) :
    print(f)

Champs contenant du texte :
content.sections.subsections
content.sections.subsections.subsections
content.sections.subsections.text
content.sections.subsections.title
content.sections.text
content.sections.title
content.summary
content.title
url


In [115]:
with open("data/qrels.json", "r", encoding="utf-8") as f :
    qrels = json.load(f)
results = {} 

for query_text in qrels.keys() :
    res = es.search(
        index="wikipedia",
        query={
            "multi_match" : {
                "query": query_text,
                "fields": list(text_fields),
                "type": "most_fields" # best_fields
            }
        },
        size=100
    )

    results[query_text] = [(hit["_id"], hit["_score"]) for hit in res["hits"]["hits"]]

In [116]:
for query, hits in results.items() :
    print(f"\nQuery : {query}")
    for i, (doc_id, score) in enumerate(hits[:1], start=1) :
        print(f"{i}. {doc_id} - score: {score}")


Query : long cutscenes stealth game with radio conversations
1. 99Iu1tHHYH - score: 35.73718

Query : open world parkour across historical city rooftops
1. DElQDCBGfl - score: 53.678482

Query : top down handheld adventure with dungeons and keys
1. vxYq433g1j - score: 40.999847

Query : scientist in hazmat suit escaping a research facility after an experiment goes wrong
1. KHWgUvytOU - score: 57.331432

Query : neo geo arcade shooter known for heavy machine gun
1. 9TSl9_dOiL - score: 70.57631

Query : cartoon platformer with a limbless hero and floating hands
1. Uf6uvvkzYD - score: 38.849888

Query : life simulation game where you build a house and control a family
1. lkkB45uZYm - score: 62.0547

Query : train and race chocobos to find hidden treasures
1. lkkB45uZYm - score: 51.913616

Query : open world crime sandbox where you can steal any car
1. 9XbEWrFEKu - score: 45.790924

Query : super soldier program in nazi fortified castle
1. vH5wxUbmMx - score: 44.956097

Query : blast demo

### Query Baseline 

In [124]:
with open("data/qrels.json", "r", encoding="utf-8") as f :
    qrels = json.load(f)
results = {} 

for query_text in qrels.keys() :
    res = es.search(
        index="wikipedia",
        query={
            "multi_match" : {
                "query": query_text,
                "fields": list(text_fields),
            }
        },
        size=100
    )

    results[query_text] = [(hit["_id"], hit["_score"]) for hit in res["hits"]["hits"]]

In [125]:
for query, hits in results.items() :
    print(f"\nQuery : {query}")
    for i, (doc_id, score) in enumerate(hits[:2], start=1) :
        print(f"{i}. {doc_id} - score: {score}")


Query : long cutscenes stealth game with radio conversations
1. uKPiWHNGc0 - score: 15.488652
2. 7rlOsfL4rN - score: 15.488652

Query : open world parkour across historical city rooftops
1. DElQDCBGfl - score: 21.346376
2. -QpOStAVai - score: 21.346376

Query : top down handheld adventure with dungeons and keys
1. 0UkVv73Cfw - score: 16.344856
2. mX4FJWTtIf - score: 15.390259

Query : scientist in hazmat suit escaping a research facility after an experiment goes wrong
1. KHWgUvytOU - score: 23.967096
2. Ibj0PzYySF - score: 20.68229

Query : neo geo arcade shooter known for heavy machine gun
1. wTimBrFPeT - score: 27.119688
2. UeKNHSHB2z - score: 26.945675

Query : cartoon platformer with a limbless hero and floating hands
1. htLsZxpCO5 - score: 16.14294
2. jvHMtHpx67 - score: 16.096125

Query : life simulation game where you build a house and control a family
1. 9b1gZBb3nQ - score: 24.256336
2. lkkB45uZYm - score: 23.826632

Query : train and race chocobos to find hidden treasures
1. 

### Analyse des resultats - Metrics

In [126]:
predictions = {}
for query, hits in results.items() :
    predictions[query] = {doc_id: float(score) for doc_id, score in hits}

In [127]:
metrics = ['P_10', 'P_100', 'recall_100', 'ndcg_cut_100']

evaluator = pytrec_eval.RelevanceEvaluator(qrels, metrics)
eval_results = evaluator.evaluate(predictions)

all_precision_10 = []; all_precision_100 = []; all_recall = []; all_dcg = []

for metrics in eval_results.values():
    all_precision_10.append(metrics['P_10'])
    all_precision_100.append(metrics['P_100'])
    all_recall.append(metrics['recall_100'])
    all_dcg.append(metrics['ndcg_cut_100'])

In [128]:
def dcg_at_k(hits, qrels, k=100):
    dcg = 0.0
    for i, doc_id in enumerate(hits[:k], start=1):
        rel = qrels.get(doc_id, 0)
        dcg += rel / math.log2(i + 1) 
    return dcg

def hit_rate_at_k(qrels, results, k=100):
    hits_count = 0
    total_queries = len(qrels)
    for query, rel_docs in qrels.items():
        top_k_docs = [doc_id for doc_id, _ in results.get(query, [])][:k]
        if any(doc_id in rel_docs and rel_docs[doc_id] > 0 for doc_id in top_k_docs):
            hits_count += 1
    return hits_count / total_queries

dcg_scores = {}
for query, hits in results.items() :
    hit_ids = [doc_id for doc_id, _ in hits]
    dcg_scores[query] = dcg_at_k(hit_ids, qrels[query], k=100)

avg_dcg = sum(dcg_scores.values()) / len(dcg_scores)
hit_rate_100 = hit_rate_at_k(qrels, results, k=100)

In [129]:
print("Moyenne Precision@10 : ", round(sum(all_precision_10)/len(all_precision_10), 2))
print("Moyenne Precision@100 : ", round(sum(all_precision_100)/len(all_precision_100), 2))
print("Moyenne Recall@100 : ", round(sum(all_recall)/len(all_recall), 2))
print("Moyenne NDCG@100 : ", round(sum(all_dcg)/len(all_dcg), 2))
print("Moyenne DCG@100 : ", round(avg_dcg, 2))
print("Hit Rate@100 : ", round(hit_rate_100, 2))

Moyenne Precision@10 :  0.23
Moyenne Precision@100 :  0.06
Moyenne Recall@100 :  0.49
Moyenne NDCG@100 :  0.29
Moyenne DCG@100 :  16.44
Hit Rate@100 :  1.0


### Observation

#### Moyenne Precision@100 :  0.06 et Precision@10 :  0.23
- Représente le rapport du nombre de documents dans le top 100 qui sont pertinents par rapport au docs dans le ground truth (GT, qrels).
- Valeur faible, cela s’explique en partie par le fait que le GT contient un nombre fini de documents pertinents.
- Exemple : pour la première requête, il y a 14 documents pertinents dans le GT. La précision maximale possible serait 14%.
- J'ai donc essayé de cacluler la précision Precision@10 qui donne 0.23, ce qui montre que le moteur de recherche met une part de documents pertinents dans les 10 premiers résultats.

#### Moyenne Recall@100 :  0.49
- Représente la proportion des documents pertinents (GT) qui sont retrouvés dans le top 100.
- Ici la recherche couvre en moyenne 49% des documents pertinents.

#### Moyenne NDCG@100 :  0.29
- Mesure la qualité du classement des documents pertinents dans le top 100, normalisée entre 0 et 1.
- La valeur faible (0.29) indique que les documents pertinents qui apparaissent ne sont pas bien positionnés dans le classement.

#### Moyenne DCG@100 :  16.44

- Correspond a la somme des gains (= niveaux de pertinence) pondérés par la position dans la liste.
- NCDG est plus représenttif car il est normalisé. Ici la valeur reste faible et indique que les documents pertinents apparaissent loin dans le classement.

#### Hit Rate@100 :  1.0

- Represente la proportion du nbr de requêtes pour lesquelles au moins un document du GT apparaît dans le top 100.

- Ici toutes les requêtes ont au moins un document pertinent dans le top 100. le moteur retrouve toujours quelque chose de pertinent, même si ce n’est pas très précis ni bien classé.

#### Synthèse globales

Les documents pertinents sont bien retrouvés mais mal classés.
Le rappel est correct (49%), ce qui montre assez une bonne couverture et aucune requête ne renvoie un top 100 complètement vide de document pertinent (GT) ce qui est rassurant.

### Conclusion  
- Le moteur de recherche retrouve quand même des documents pertinents. 
- Le ranking doit clairement être amélioré. 
- Une seconde amélioration peut être liée à la façon d'indexer les documents. Ici les données sont indexées champ par champ, une seconde version pourrait consister à flatten les données pour donner plus de contexte aux requêtes

## Partie 2 — Extensions et experimentations

### Ajout du LLM pour rerank

In [ ]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [138]:
def llm_evaluate(query : str, document_text : str) :
    """
    Utilise llm pour rerank via un analyse query by query. 
    """
    prompt = (
        f"Évalue la pertinence du document par rapport à la requête.\n"
        f"Requête : {query}\n"
        f"Document : {document_text}\n"
        f"Renvoie uniquement un objet JSON avec un seul champ :\n"
        f"{{\"score\": x.xxx}}\n"
        f"Le score doit être un nombre entre 0 et 1, avec 3 ou 4 décimales, "
        f"sans texte supplémentaire."
    )

    try:
        response = client.responses.create(
            model="gpt-4",  # À des fins de réduire les couts comparé à GPT 5
            input=prompt,
            temperature=0
        )
        text = response.output[0].content[0].text.strip()
        data = json.loads(text)
        score = float(data["score"])
        return max(0.0, min(1.0, score))
    
    except Exception as e :
        print("Erreur parsing JSON ou LLM:", e)
        return 0.0

In [139]:
# test
query = "long cutscenes stealth game with radio conversations"
document_text = "This game features long stealth missions with radio conversations and multiple objectives."
score = llm_evaluate(query, document_text)
print(score)

0.875


In [142]:
# Reduction du texte en input du LLM et calcul du nombre de token pour tronquer.
MODEL_NAME = "gpt-4" 
ENC = tiktoken.encoding_for_model(MODEL_NAME)

def extract_title_summary(doc, max_tokens=2000) :
    """
    Extrait titre et summary
    """
    parts = []
    if "content" in doc:
        if "title" in doc["content"]:
            parts.append(f"Title : {doc['content']['title']}")
        if "summary" in doc["content"]:
            parts.append(f"Summary : {doc['content']['summary']}")

    
    text = "\n".join(parts)
    tokens = ENC.encode(text)
    n_tokens = len(tokens)
    # Tronquer
    if n_tokens > max_tokens:
        truncated_tokens = tokens[:max_tokens]
        text = ENC.decode(truncated_tokens)
        print(f"[Warning] Texte tronqué : {n_tokens} → {max_tokens} tokens")
    
    return text

In [143]:
with mlflow.start_run(run_name="BM25 + LLM reranking"):
    mlflow.log_param("top_k", 100)
    mlflow.log_param("use_llm_reranking", True)
    
    results = {}

    # Requete comme precedemment
    qrels_subset = dict(list(qrels.items())[:3]) # Pour limiter le temps d'execution on prend juste les top 3 query
    for query_text in qrels_subset.keys():
        res = es.search(
            index="wikipedia",
            query={
                "multi_match": {
                    "query": query_text,
                    "fields": list(text_fields),
                }
            },
            size=100
        )
        docs = [(hit["_id"], hit["_source"]) for hit in res["hits"]["hits"]]

        # Ajout du score via LLM
        scored_docs = []
        for doc_id, text in docs :
            llm_input_text = extract_title_summary(text)
            score = llm_evaluate(query_text, llm_input_text)
            scored_docs.append((doc_id, score))
        reranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
        results[query_text] = reranked_docs

    # Calcul de metriques
    metrics = ['P_10', 'P_100', 'recall_100', 'ndcg_cut_100']
    evaluator = pytrec_eval.RelevanceEvaluator(qrels, metrics)
    eval_results = evaluator.evaluate(predictions)

    all_precision_10 = []; all_recall = []; all_ndcg = []; all_dcg = []

    for query, query_metrics in eval_results.items() :
        all_precision_10.append(query_metrics['P_10'])
        all_precision_100.append(query_metrics['P_100'])
        all_recall.append(query_metrics['recall_100'])
        all_ndcg.append(query_metrics['ndcg_cut_100'])

        top_100_docs = list(predictions[query].keys())[:100]
        all_dcg.append(dcg_at_k(top_100_docs, qrels[query], k=100))

    mlflow.log_metric("precision_10", sum(all_precision_10)/len(all_precision_10))
    mlflow.log_metric("precision_100", sum(all_precision_100)/len(all_precision_100))
    mlflow.log_metric("recall_100", sum(all_recall)/len(all_recall))
    mlflow.log_metric("ndcg_100", sum(all_ndcg)/len(all_ndcg))
    mlflow.log_metric("dcg_100", sum(all_dcg)/len(all_dcg))

    print("Moyenne Precision@10 : ", round(sum(all_precision_10)/len(all_precision_10), 2))
    print("Moyenne Precision@100 : ", round(sum(all_precision_100)/len(all_precision_100), 2))
    print("Moyenne Recall@100 : ", round(sum(all_recall)/len(all_recall), 2))
    print("Moyenne NDCG@100 : ", round(sum(all_dcg)/len(all_dcg), 2))
    print("Moyenne DCG@100 : ", round(avg_dcg, 2))

mlflow.end_run()

Moyenne Precision@10 :  0.23
Moyenne Precision@100 :  0.06
Moyenne Recall@100 :  0.49
Moyenne NDCG@100 :  16.44
Moyenne DCG@100 :  16.44


### Conclusion
- Les performances n’ont pas progressées (ceci est en lien avec la façon dont le rerank via LLM as a judge est mise en place).
- Limite identifiée : le LLM, appelé pour chaque document individuellement, ne tient pas compte de l’ensemble des résultats (il ne compare pas les documents entre eux mais calcule le score un à un).
- Une explication détaillé est présente dans le rapport, ainsi que les améliorations à mettre en place.

In [145]:
# Afin de voir les résultats dans mlflow, run la commande suivante dans le terminal :
# poetry run mlflow ui